In [62]:
import numpy as np

In [116]:
class GeneticAOTSP(object):
    # constante pentru setarea algoritmului
    POPULATION_SIZE = 100 # numarul populatiei
    GENOME_LENGTH   = 4 # numarul de orase
    MUTATION_RATE   = 0.01  # threshold-ul pentru a face o mutatie genetica
    CROSSOVER_RATE  = 0.5   # threshold-ul pentru incrucisarea parintilor
    SELECT_RATE     = 0.8   # threshold-ul de selectie, selectare dupa compatibilitate sau dupa probabilitate
    SELECT_K        = 0.1   # coeficientul de selectie, devierea minus plus de la valoarea fitness
    GENERATIONS     = 500 # numarul de generatii

    def __init__(self, fitness_fn=None):
        if (fitness_fn is not None):
            self.fitness = fitness_fn
        else:
            self.fitness = self.__fitness

    def __call__(self, distance:"np.array"):
        # save map
        if (distance is not None):
            self.distance = distance
            self.total_distance = np.sum(self.distance, axis=None)
            # update numarul de orase
            GeneticAOTSP.GENOME_LENGTH = self.distance.shape[0]
        # initiaizarea populatiei
        population = self.initPopulation()
        print("population", population)
        # init fitness value
        fitness_values = self.fitness(population)
        # evolutia generatiilor
        for generation in range(GeneticAOTSP.GENERATIONS):
            # nasterea unei noi generatii
            new_population = []
            for _ in range(GeneticAOTSP.POPULATION_SIZE):
                # selectarea parintilor
                arg_parent1 = self.selectParent1(population, fitness_values)
                arg_parent2 = self.selectParent2(population, fitness_values, arg_parent1)
                parent1 = population[arg_parent1]
                parent2 = population[arg_parent2]
                # incrucisarea parintilor
                child = self.crossover(parent1, parent2)
                # mutatii
                child = self.mutate(child)
                new_population.append(child)
            # schimbarea generatiei
            population = np.array(new_population, dtype=np.int32)
            # init fitness value
            fitness_values = self.fitness(population)
            # selectarea celei mai bune rute
            bestRoute = self.getBestRoute(population)
            bestDistance = self.calculateDistance(bestRoute)
            print("Generatia: {}, Cea mai scurta cale: {}, Distanta: {}".format(
                generation, bestRoute, bestDistance
                ))

    def initPopulation(self):
        """Initializarea populatiei, cu drumuri aleatorii"""
        size = (GeneticAOTSP.POPULATION_SIZE, GeneticAOTSP.GENOME_LENGTH)
        arr = np.arange(np.prod(size), dtype=np.int32).reshape(*size)%GeneticAOTSP.GENOME_LENGTH
        population = np.apply_along_axis(np.random.permutation, axis=1, arr=arr)
        return population

    def selectParent1(self, population, fitness_values):
        """selectarea unui parinte aleator din populatie, bazandune pe distributia fitness valorilor"""
        # select random parent
        prob_fitness = fitness_values / fitness_values.sum()
        arg = np.random.choice(GeneticAOTSP.POPULATION_SIZE, size=None, p=prob_fitness)
        return arg

    def selectParent2(self, population, fitness_values, arg_partener):
        """selectarea unui parinte aleator din populatie"""
        select_rate = np.random.uniform(low=0, high=1, size=None)
        if (select_rate < GeneticAOTSP.SELECT_RATE): # selectie dupa compatibilitate
            fitness_partener = fitness_values[arg_partener]
            start_fitness = fitness_partener * (1 - GeneticAOTSP.SELECT_K)
            stop_fitness  = fitness_partener * (1 + GeneticAOTSP.SELECT_K)
            start_arg = fitness_values > start_fitness
            stop_arg  = fitness_values < stop_fitness
            arg = start_arg&stop_arg
            tmp_fitness_values = fitness_values.copy()
            inv_arg = np.logical_not(arg)
            tmp_fitness_values[arg] = 0.
            prob_fitness = tmp_fitness_values / tmp_fitness_values.sum()
            print("selectie dupa compatibilitate", prob_fitness)
        else: # selectie aleatorie
            prob_fitness = fitness_values / fitness_values.sum()
            print("selectie aleatorie", prob_fitness)
        # selecteaza argumentul parintelui 2
        arg = np.random.choice(GeneticAOTSP.POPULATION_SIZE, size=None, p=prob_fitness)
        return arg

    def crossover(self, parent1, parent2):
        """Incrucisarea a doi parinti pentru a crea un urmas
        """
        # creare un copil fara mostenire
        child = np.zeros(parent1.shape[0])
        # selectarea diapazonului de mostenire
        start = np.random.randint(low=0, high=parent1.shape[0], size=None)
        end   = np.random.randint(low=0, high=parent1.shape[0], size=None)
        if (start > end):
            start, end = end, start
        # copierea rutei din primul parinte
        child[start:end] = parent1[start:end]
        # copierea rutei din cel de al doilea parinte
        child[:start] = parent2[:start]
        child[end:]   = parent2[end:]
        return child

    def mutate(self, individ):
        """Mutatia genetica a rutei"""
        # selectarea genomurile care vor fi mutate cu locul
        cond = np.random.uniform(low=0, high=1, size=None)
        if (cond < GeneticAOTSP.MUTATION_RATE):
            index1 = np.random.randint(low=0, high=individ.shape[0], size=None)
            index2 = np.random.randint(low=0, high=individ.shape[0], size=None)
            individ[index1], individ[index2] = individ[index2], individ[index1]
        return individ

    def calculateDistance(self, individ):
        """Calculul distantei rutelor"""
        #print("individ", individ)
        distance = self.distance[individ[:-1], individ[1:]].sum()
        distance += self.distance[individ[-1], individ[0]]
        return distance

    def getBestRoute(self, population):
        """Cautarea rutei optime din populatie"""
        distances = self.__getDistances(population)
        index = np.argmin(distances, axis=None, keepdims=False)
        return population[index]

    def __getDistances(self, population):
        """calcularea distantei pentru fiecare individ din populatiei"""
        return np.apply_along_axis(self.calculateDistance,
                                        axis=1,
                                        arr=population)

    def __fitness(self, population):
        """ valoarea la fitness este inversul distantei dintre orase si un numar cat mai mare de orase vizitate
        - se calculeaza distanta maxima pentru fiecare individ
        - se calculeaza numarul de orase unice
        """
        # calculeaza distanta
        distances = self.__getDistances(population)
        arg_zero = distances == 0
        distances[arg_zero] = -1
        max_distance = np.max(distances)
        # calculeaza numarul de orase unice
        cout_unique_fn = lambda individ: np.unique(individ, return_index=False, return_inverse=False, return_counts=False, axis=None).shape[0]
        cout_unique = np.apply_along_axis(cout_unique_fn,
                                        axis=1,
                                        arr=population)
        # calculeaza coeficientul
        if (max_distance == 0):
            distances = distances + 0.0001
        else:
            distances = max_distance/(distances + 0.0001)
        fitness_values = distances*(cout_unique/GeneticAOTSP.GENOME_LENGTH)
        return fitness_values

    def setParameters(self, **kw):
        GeneticAOTSP.POPULATION_SIZE = kw.get("POPULATION_SIZE", GeneticAOTSP.POPULATION_SIZE)
        GeneticAOTSP.GENOME_LENGTH   = kw.get("GENOME_LENGTH", GeneticAOTSP.GENOME_LENGTH)
        GeneticAOTSP.MUTATION_RATE   = kw.get("MUTATION_RATE", GeneticAOTSP.MUTATION_RATE)
        GeneticAOTSP.CROSSOVER_RATE  = kw.get("CROSSOVER_RATE", GeneticAOTSP.CROSSOVER_RATE)
        GeneticAOTSP.SELECT_RATE     = kw.get("SELECT_RATE", GeneticAOTSP.SELECT_RATE)
        GeneticAOTSP.SELECT_K        = kw.get("SELECT_K", GeneticAOTSP.SELECT_K)
        GeneticAOTSP.GENERATIONS     = kw.get("GENERATIONS", GeneticAOTSP.GENERATIONS)



In [65]:
map_of_distance = [
    [0, 10, 15, 25],
    [10, 0, 20, 35],
    [15, 20, 0, 30],
    [25, 35, 30, 0]]
map_of_distance = np.array(map_of_distance)

In [117]:
tsp = GeneticAOTSP()

In [118]:

tsp.setParameters(
    POPULATION_SIZE = 100, # numarul populatiei
    GENOME_LENGTH   = 4,   # numarul de orase
    MUTATION_RATE   = 0.01,  # threshold-ul pentru a face o mutatie genetica
    CROSSOVER_RATE  = 0.5,   # threshold-ul pentru incrucisarea parintilor
    SELECT_RATE     = 0.8,   # threshold-ul de selectie, selectare dupa compatibilitate sau dupa probabilitate
    SELECT_K        = 0.4,   # coeficientul de selectie, devierea minus plus de la valoarea fitness
    GENERATIONS     = 500,  # numarul de generatii
    )
tsp(map_of_distance)

population [[1 0 2 3]
 [0 1 3 2]
 [2 1 3 0]
 [0 3 1 2]
 [0 2 3 1]
 [0 2 1 3]
 [3 0 1 2]
 [1 2 0 3]
 [1 3 2 0]
 [3 0 1 2]
 [1 0 3 2]
 [3 0 2 1]
 [0 2 1 3]
 [1 0 2 3]
 [2 3 1 0]
 [3 1 0 2]
 [3 2 0 1]
 [0 3 2 1]
 [1 0 3 2]
 [1 0 2 3]
 [3 1 0 2]
 [1 3 0 2]
 [0 3 2 1]
 [1 3 2 0]
 [1 3 2 0]
 [3 1 0 2]
 [2 0 1 3]
 [3 1 0 2]
 [1 3 0 2]
 [0 2 1 3]
 [0 3 2 1]
 [1 3 0 2]
 [2 1 3 0]
 [3 0 2 1]
 [1 0 2 3]
 [1 2 0 3]
 [1 2 3 0]
 [2 1 3 0]
 [1 0 2 3]
 [0 1 3 2]
 [3 0 2 1]
 [3 0 2 1]
 [2 1 0 3]
 [1 0 2 3]
 [1 2 0 3]
 [3 0 2 1]
 [3 1 2 0]
 [2 0 1 3]
 [1 3 0 2]
 [0 2 1 3]
 [1 0 2 3]
 [0 2 1 3]
 [2 3 1 0]
 [0 1 3 2]
 [0 3 2 1]
 [0 1 3 2]
 [3 2 0 1]
 [1 2 3 0]
 [0 3 1 2]
 [1 0 3 2]
 [0 3 1 2]
 [1 0 3 2]
 [3 2 0 1]
 [0 3 2 1]
 [1 0 2 3]
 [1 3 0 2]
 [2 3 0 1]
 [2 0 3 1]
 [2 3 0 1]
 [0 3 2 1]
 [1 0 3 2]
 [3 2 1 0]
 [0 3 2 1]
 [3 1 0 2]
 [0 1 3 2]
 [1 2 3 0]
 [2 1 0 3]
 [3 2 0 1]
 [1 0 3 2]
 [2 1 0 3]
 [3 2 1 0]
 [2 0 1 3]
 [2 3 0 1]
 [2 3 1 0]
 [2 0 1 3]
 [1 2 3 0]
 [0 3 2 1]
 [2 0 3 1]
 [0 2 1 3]
 [0 2 1 3]

/tmp/ipython-input-4051555299.py:82: RuntimeWarning: invalid value encountered in divide
  prob_fitness = tmp_fitness_values / tmp_fitness_values.sum()


ValueError: probabilities contain NaN

In [6]:
distance = np.sum(population, axis=1)
np.argmin(distance, axis=None, keepdims=False)

np.int64(0)

In [23]:
map_of_distance[[0, 1],[0, 3]]

array([ 0, 35])

In [61]:

tmp = np.random.randint(low=0, high=30, size=(2, 6))
print(tmp)
#np.unique(tmp, return_index=False, return_inverse=False, return_counts=False, axis=None)
uniques = [np.unique(individ, return_index=False, return_inverse=False, return_counts=False, axis=None) for individ in tmp]
print(uniques)
np.apply_along_axis(lambda individ: np.unique(individ, return_index=False, return_inverse=False, return_counts=False, axis=None).shape[0],
                                        axis=1,
                                        arr=tmp)

[[ 0  5 15 24 24 20]
 [13  5 26 13 16 12]]
[array([ 0,  5, 15, 20, 24]), array([ 5, 12, 13, 16, 26])]


array([5, 5])